In [12]:
import requests
import json
from datetime import datetime, timedelta
import csv

In [9]:
api_key = "SjCCIumG56fAPhvxnaQiFLHz05hwae8iVLgzcYPs"

# Function to fetch data for a single week
def fetch_neo_data(start_date, end_date, api_key):
    url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve data for {start_date} to {end_date}. Status code: {response.status_code}")
        return None

In [10]:
start_date = datetime.strptime("2023-02-06", '%Y-%m-%d')
end_date = datetime.strptime("2024-02-06", '%Y-%m-%d')
current_date = start_date

# Container for all data
all_data = []

while current_date < end_date:
    week_end_date = current_date + timedelta(days=7)
    if week_end_date > end_date:
        week_end_date = end_date
    
    # Fetch data for the current week
    data = fetch_neo_data(current_date.strftime('%Y-%m-%d'), week_end_date.strftime('%Y-%m-%d'), api_key)
    if data:
        all_data.append(data)
    
    # Prepare for the next iteration
    current_date = week_end_date + timedelta(days=1)

# Combine all data and save to file (This data is by week)
with open('neo_data_year.json', 'w') as outfile:
    json.dump(all_data, outfile, indent=4)

print("Data for the entire year retrieved and saved successfully.")

Data for the entire year retrieved and saved successfully.


In [15]:
with open('neo_data_year.json', 'r') as file:
    all_neo_data = json.load(file)  


csv_columns = [
    'Date', 'ID', 'Name', 'Neo Reference ID', 'NASA JPL URL', 'Absolute Magnitude H',
    'Estimated Diameter Min (km)', 'Estimated Diameter Max (km)', 'Is Potentially Hazardous Asteroid',
    'Close Approach Date Full', 'Rela]ive Velocity (km/h)', 'Miss Distance (astronomical units)',
    'Miss Distance (kilometers)', 'Orbiting Body'
]

csv_file = "neo_data_year.csv"


with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()

    # Iterate through each week's data
    for weekly_data in all_neo_data:
       
        for date, neos in weekly_data['near_earth_objects'].items():
            for neo in neos:
                row = {
                    'Date': date,
                    'ID': neo.get('id', 'N/A'),
                    'Name': neo.get('name', 'N/A'),
                    'Neo Reference ID': neo.get('neo_reference_id', 'N/A'),
                    'NASA JPL URL': neo.get('nasa_jpl_url', 'N/A'),
                    'Absolute Magnitude H': neo.get('absolute_magnitude_h', 'N/A'),
                    'Estimated Diameter Min (km)': neo.get('estimated_diameter', {}).get('kilometers', {}).get('estimated_diameter_min', 'N/A'),
                    'Estimated Diameter Max (km)': neo.get('estimated_diameter', {}).get('kilometers', {}).get('estimated_diameter_max', 'N/A'),
                    'Is Potentially Hazardous Asteroid': neo.get('is_potentially_hazardous_asteroid', 'N/A'),
                    'Close Approach Date Full': neo.get('close_approach_data', [{}])[0].get('close_approach_date_full', 'N/A'),
                    'Relative Velocity (km/h)': neo.get('close_approach_data', [{}])[0].get('relative_velocity', {}).get('kilometers_per_hour', 'N/A'),
                    'Miss Distance (astronomical units)': neo.get('close_approach_data', [{}])[0].get('miss_distance', {}).get('astronomical', 'N/A'),
                    'Miss Distance (kilometers)': neo.get('close_approach_data', [{}])[0].get('miss_distance', {}).get('kilometers', 'N/A'),
                    'Orbiting Body': neo.get('close_approach_data', [{}])[0].get('orbiting_body', 'N/A')
                }
                writer.writerow(row)

print("CSV file has been created successfully.")

CSV file has been created successfully.
